<a href="https://colab.research.google.com/github/log-ghj/automatic-model-selection/blob/main/ridge_CV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import Ridge
import scipy
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import PolynomialFeatures

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
#Read the data
df = pd.read_csv('https://raw.githubusercontent.com/ageron/handson-ml/master/datasets/housing/housing.csv').dropna()

#Convert number of rooms into per person values
df["total_rooms_pp"] = df["total_rooms"]/df["population"]
df["total_bedrooms_pp"] = df["total_bedrooms"]/df["population"]
#Average hosuehold size
df["household_size"] = df["population"]/df["households"]
#Drop some variables
df=df.drop(["total_rooms", "total_bedrooms", "households"], axis=1)

#Make the categorical variable into a set of dummies
xx = pd.get_dummies(df.ocean_proximity)
df = pd.concat([df, xx], axis=1, sort=False)
del df["ocean_proximity"]

In [3]:
#Cluster geolocations - determine the optimal number of clusters
from sklearn import preprocessing, cluster
X = df[["latitude","longitude"]]
max_k = 10
## iterations
distortions = [] 
for i in range(1, max_k+1):
    if len(X) >= i:
       model = cluster.KMeans(n_clusters=i, init='k-means++', max_iter=300, n_init=10, random_state=0)
       model.fit(X)
       distortions.append(model.inertia_)
## best k: the lowest derivative
k = [i*100 for i in np.diff(distortions,2)].index(min([i*100 for i 
     in np.diff(distortions,2)]))

In [4]:
#Cluster geolocations - create clusters
import scipy.cluster
k = 6
model = cluster.KMeans(n_clusters=k, init='k-means++')
X = df[["latitude","longitude"]]
# clustering
df_X = X.copy()
df_X["cluster"] = model.fit_predict(X)
# find real centroids
closest, distances = scipy.cluster.vq.vq(model.cluster_centers_, df_X.drop("cluster", axis=1).values)
df_X["centroids"] = 0
for i in closest:
    df_X["centroids"].iloc[i] = 1
# add clustering info to the original dataset
df[["cluster"]] = df_X[["cluster"]]

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [5]:
from sklearn.base import BaseEstimator 
from sklearn.base import RegressorMixin

In [6]:
class self_coded_ridge(BaseEstimator, RegressorMixin):
    def __init__(self, λ=1, **model_hyper_parameters):
        """
        """
        super().__init__()
        self.λ = λ
        self.X_train = None
        # fitted parameters, initialized to None
        self.params = None

    def set_params(self, **params):
        """
        """
        if not params:
            # Simple optimization to gain speed (inspect is slow)
            return self
        valid_params = self.get_params(deep=True)

        nested_params = defaultdict(dict)  # grouped by prefix
        for key, value in params.items():
            key, delim, sub_key = key.partition('__')
            if key not in valid_params:
                raise ValueError('Invalid parameter %s for estimator %s. '
                                 'Check the list of available parameters '
                                 'with `estimator.get_params().keys()`.' %
                                 (key, self))

            if delim:
                nested_params[key][sub_key] = value
            else:
                setattr(self, key, value)
                valid_params[key] = value

        for key, sub_params in nested_params.items():
            valid_params[key].set_params(**sub_params)

        return self

    def fit(self, X_train, y_train):
        """
        """
        X_train, y_train, λ = np.asarray(add_constant(X_train, prepend=True)), np.asarray(y_train), self.λ 
        self.X_train = X_train
        XX = X_train.T@X_train
        β = np.linalg.inv(XX+λ*np.eye(N=len(XX)))@X_train.T@y_train
        self.params = β
        return self.params

    def predict(self, X_test):
        β = self.params
        return X_test@β

    def get_params(self, deep = False):
        return {'λ':self.λ}

In [7]:
# Define a function that does CV on a grid to find optimal value of the hyperparameter
def ridge_self_coded(grid,X,y):
  coefs = []
  ics = []

  for a in grid:
    ridge = self_coded_ridge(a)
    ridge.fit(X, y)
    coefs.append(ridge.params)
    k=10 #k-fold CV
    scores = cross_val_score(ridge, X, y, cv=k, scoring='neg_mean_squared_error')
    RMSE = np.sqrt(sum(abs(scores))/k)
    ics.append(RMSE)
  opt = ics.index(min(ics))
  return coefs[opt], grid[opt], ics[opt]

In [8]:
# Create a similar function for the pre-existing sklearn ridge for comparison
def ridge_sklearn(lambdas, X, y):
  ridge = Ridge(fit_intercept=False, normalize=False) # do not use sklearn's normalization as it divides by L2 and not std.
  coefs = []
  ics = []

  for a in lambdas:
    ridge.set_params(alpha = a)
    ridge.fit(X, y)
    coefs.append(ridge.coef_)
    k=10 #k-fold CV
    scores = cross_val_score(ridge, X, y, cv=k, scoring='neg_mean_squared_error')
    RMSE = np.sqrt(sum(abs(scores))/k)
    ics.append(RMSE)
  opt = ics.index(min(ics))
  return coefs[opt], lambdas[opt], ics[opt]

In [9]:
# create grid of lambdas
# doing what is called naive implementation in the slides -> max value of grid chosen
# efficient way is through SVD -> left to do
grid = 10**np.linspace(4,-2,100)*0.5

# split l.h.s, and r.h.s.
X = df.drop(['median_house_value'], axis=1)
y = df.median_house_value

# normalize data
X = (X-X.mean())/X.std()
y = y-y.mean()

poly = PolynomialFeatures(2)
X = poly.fit_transform(X)

In [10]:
X.shape

(20433, 120)

In [11]:
result_self_coded = ridge_self_coded(grid, X, y)
result_self_coded

(array([ 3.33033812e+03, -1.41220664e+04, -4.79302242e+03,  7.39762657e+03,
         3.47453533e+03,  5.14278220e+04,  2.48427000e+04,  2.54513730e+04,
        -4.48480008e+03,  1.01805154e+04, -1.12631989e+04, -5.17523724e+01,
        -5.53317707e+02,  1.08593797e+03, -1.51575278e+04, -4.05013795e+03,
         5.82543951e+03, -1.48222432e+03,  1.44774408e+03, -4.10896198e+03,
         9.87685644e+02, -7.01634649e+02, -7.48573704e+02,  2.37254845e+03,
        -2.88739528e+02,  3.39129653e+02, -2.36057609e+03, -9.18050673e+02,
         2.54425496e+03, -4.16592047e+03, -4.93154649e+03,  2.09021027e+03,
        -4.31075656e+03, -1.68009652e+03, -1.15629671e+03, -1.16341257e+03,
         1.94398863e+02,  1.51811035e+03,  2.71197267e+01, -3.37177514e+03,
         7.63722411e+02,  2.83610707e+03,  2.62115755e+03,  2.92185141e+03,
         1.80248254e+03,  1.64218008e+03,  9.11066623e+03, -1.49687208e+03,
        -1.42621421e+03, -1.87045620e+03, -2.12275424e+02,  5.12605499e+03,
        -8.4

In [12]:
result_sklearn = ridge_sklearn(grid, X, y)
result_sklearn

(array([ 3.33033812e+03, -1.41220664e+04, -4.79302242e+03,  7.39762657e+03,
         3.47453533e+03,  5.14278220e+04,  2.48427000e+04,  2.54513730e+04,
        -4.48480008e+03,  1.01805154e+04, -1.12631989e+04, -5.17523724e+01,
        -5.53317707e+02,  1.08593797e+03, -1.51575278e+04, -4.05013795e+03,
         5.82543951e+03, -1.48222432e+03,  1.44774408e+03, -4.10896198e+03,
         9.87685644e+02, -7.01634649e+02, -7.48573704e+02,  2.37254845e+03,
        -2.88739528e+02,  3.39129653e+02, -2.36057609e+03, -9.18050673e+02,
         2.54425496e+03, -4.16592047e+03, -4.93154649e+03,  2.09021027e+03,
        -4.31075656e+03, -1.68009652e+03, -1.15629671e+03, -1.16341257e+03,
         1.94398863e+02,  1.51811035e+03,  2.71197267e+01, -3.37177514e+03,
         7.63722411e+02,  2.83610707e+03,  2.62115755e+03,  2.92185141e+03,
         1.80248254e+03,  1.64218008e+03,  9.11066623e+03, -1.49687208e+03,
        -1.42621421e+03, -1.87045620e+03, -2.12275424e+02,  5.12605499e+03,
        -8.4

In [13]:
# instead of the above arbitrarily defined grid, find a grid with the help of svd
# from ols to ridge D**-1 is multiplied by (D**2+λ)**(-1) D**2 (1)
# df of ols are 120 (with poly) -> find λ s.t. (1) = ~120/~0
import scipy
u,s,v = scipy.linalg.svd(X)

In [14]:
sum(s**2/(s**2))

120.0

In [15]:
x1 = 1e-30
x2 = 1_000_000_000

In [16]:
print(sum(s**2/(s**2+x1)),sum(s**2/(s**2+x2)))

119.99990469236103 0.2999010958885242


In [17]:
# define a new grid with above valuees
new_grid = np.linspace(x1,x2,100)

In [18]:
new_grid = np.append(new_grid,result_self_coded[1])

In [19]:
ridge_self_coded(new_grid, X, y)

(array([ 3.33033812e+03, -1.41220664e+04, -4.79302242e+03,  7.39762657e+03,
         3.47453533e+03,  5.14278220e+04,  2.48427000e+04,  2.54513730e+04,
        -4.48480008e+03,  1.01805154e+04, -1.12631989e+04, -5.17523724e+01,
        -5.53317707e+02,  1.08593797e+03, -1.51575278e+04, -4.05013795e+03,
         5.82543951e+03, -1.48222432e+03,  1.44774408e+03, -4.10896198e+03,
         9.87685644e+02, -7.01634649e+02, -7.48573704e+02,  2.37254845e+03,
        -2.88739528e+02,  3.39129653e+02, -2.36057609e+03, -9.18050673e+02,
         2.54425496e+03, -4.16592047e+03, -4.93154649e+03,  2.09021027e+03,
        -4.31075656e+03, -1.68009652e+03, -1.15629671e+03, -1.16341257e+03,
         1.94398863e+02,  1.51811035e+03,  2.71197267e+01, -3.37177514e+03,
         7.63722411e+02,  2.83610707e+03,  2.62115755e+03,  2.92185141e+03,
         1.80248254e+03,  1.64218008e+03,  9.11066623e+03, -1.49687208e+03,
        -1.42621421e+03, -1.87045620e+03, -2.12275424e+02,  5.12605499e+03,
        -8.4

In [20]:
ridge_sklearn(new_grid, X, y)

(array([ 3.33033812e+03, -1.41220664e+04, -4.79302242e+03,  7.39762657e+03,
         3.47453533e+03,  5.14278220e+04,  2.48427000e+04,  2.54513730e+04,
        -4.48480008e+03,  1.01805154e+04, -1.12631989e+04, -5.17523724e+01,
        -5.53317707e+02,  1.08593797e+03, -1.51575278e+04, -4.05013795e+03,
         5.82543951e+03, -1.48222432e+03,  1.44774408e+03, -4.10896198e+03,
         9.87685644e+02, -7.01634649e+02, -7.48573704e+02,  2.37254845e+03,
        -2.88739528e+02,  3.39129653e+02, -2.36057609e+03, -9.18050673e+02,
         2.54425496e+03, -4.16592047e+03, -4.93154649e+03,  2.09021027e+03,
        -4.31075656e+03, -1.68009652e+03, -1.15629671e+03, -1.16341257e+03,
         1.94398863e+02,  1.51811035e+03,  2.71197267e+01, -3.37177514e+03,
         7.63722411e+02,  2.83610707e+03,  2.62115755e+03,  2.92185141e+03,
         1.80248254e+03,  1.64218008e+03,  9.11066623e+03, -1.49687208e+03,
        -1.42621421e+03, -1.87045620e+03, -2.12275424e+02,  5.12605499e+03,
        -8.4